In [2]:
%load_ext autoreload
%autoreload 2

from ble import get_ble_controller
from base_ble import LOG
from cmd_types import CMD
from funcs import *
from robotClass import *

import time
import numpy as np
import matplotlib.pyplot as plt
import asyncio

LOG.propagate = False

### Lab 8


In [4]:
# Setup robot
rc = setupRobot()

2022-03-23 15:41:12,495 | INFO     |: Looking for Artemis Nano Peripheral Device: C0:83:B0:6C:9A:3C
2022-03-23 15:41:15,488 | INFO     |: Connected to C0:83:B0:6C:9A:3C


In [6]:
rc.pingRobot(clear = True, startX = 500)

2022-03-23 15:41:42,101 | INFO     |: Disconnected from 607452C2-8441-D076-814E-841EB3635426


In [ ]:
# Ping robot to start writing PWM values to the characteristic
rc.pingRobot(clear = True) #startX = 1000
rc.updatePID(350, 0.05, 0, 0.75, 1000, 200)

# Wait one second
await asyncio.sleep(2)

# Apply a constant PWM signal for enough time to reach steady state (constant speed)
rc.move_forward([50, 50], 0, 0) # PWM: 100, 0: forward, 0: do PID (no PWM adjustment)
await asyncio.sleep(10) # start with 5 seconds but make it longer if needed
rc.stop()
await asyncio.sleep(2)

# Ping robot to stop writing PWM values to the characteristic
rc.pingRobot()

In [ ]:
# Get front ToF sensor output
frontTOFData = rc.tof2_readings
startTimeTOF = frontTOFData[0][1]

# Get KF ToF Sensor output
kfTOF = rc.kf_tof
startTimeKFTOF = kfTOF[0][1]

# Get speed data
# format: [ [velocities], [times] ]
speedData = distToVelocity(frontTOFData)

# Get motor PWM input
motorData = rc.motor_readings

# Get KF motor PWM
kfMotorData = rc.kf_motor_pwm

In [ ]:
# Plot ---

# Front ToF Sensor Output
plotTOFData = dataForPlot(frontTOFData)
plotKFTOFData = dataForPlot(kfTOF)

plt.plot([x - startTimeTOF for x in plotTOFData[1]], plotTOFData[0], 'ro')
plt.plot([x - startTimeTOF for x in plotKFTOFData[1]], plotKFTOFData[0])

plt.title('Kalman Filter vs. Front ToF Sensor')
plt.xlim([0, 20])
plt.xlabel('Time (sec)')
plt.ylabel('Distance (mm)')
plt.show()

# Motor data
plotMotorData = dataForPlot(motorData)
startTimeMotor = plotMotorData[1][0]
plt.plot([x - startTimeMotor for x in plotMotorData[1]], plotMotorData[0])
plt.title('PWM Input Graph')
plt.xlabel('Time (sec)')
plt.ylabel('PWM Signal Input')
plt.show()